In [1]:
import pyodbc
import pandas as pd
import numpy as np
from scipy import optimize
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
class DBConnector:
    def __init__(self, server_name, username, password, db_name, connection_str):
        self.sever_name = server_name
        self.username = username
        self.password = password
        self.db_name = db_name
        self.connection_str = connection_str
        self.connection = pyodbc.connect(self.connection_str)

    def test_connection(self):
        try:
            db = self.connection
            cursor = db.cursor()
            cursor.execute("SELECT @@VERSION")
            results = cursor.fetchone()
            # Check if anything at all is returned
            if results:
                return True
            else:
                return False
        except pyodbc.Error as ex:
            print(ex)
            print("Error in connection!")
            sqlstate = ex.args[0]
            if sqlstate == '28000':
                print("LDAP Connection failed: check password")
        return False

    def query(self, query_str):
        """This function returns a dataframe of result list"""
        return pd.read_sql_query(query_str, self.connection)

    def all_columns_name(self, table_name):
        """This function returns a dataframe of result list"""
        query_str = f"select COLUMN_NAME from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='{table_name}'"
        print("All columns names: ", query_str)
        return pd.read_sql_query(query_str, self.connection)

driver="SQL Server"
servername = 'QUOC-CUONG'
username = 'sa'
password = 'cuong300599'
db_name = 'OnlinePhoneShopJoin'

str_for_connection = "Driver={driver};Server={servername};UID={username};PWD={password};Database={db_name};"\
    .format(driver=driver, servername=servername, username=username, password=password, db_name=db_name)

connector = DBConnector(servername, username, password, db_name, str_for_connection)

In [3]:
query_str = "select distinct u.user_id, p.product_id, dbo.fn_get_rating(u.user_id, p.product_id) as score from users u, products p"
df_users_products_ratings = connector.query(query_str)
df_users_products_ratings_pivotted = df_users_products_ratings.pivot(index='product_id', columns='user_id', values='score').fillna(0)
print("Number of products (rows): ", df_users_products_ratings_pivotted.shape[0])
print("Number of  users (columns): ", df_users_products_ratings_pivotted.shape[1])

# Save to csv file
df_users_products_ratings_pivotted.to_csv(r'D:\PhoneShopML\data\rating_matrix.csv', header=True, index=False, encoding='utf-8-sig')

df_users_products_ratings_pivotted.head(20)

Number of products (rows):  61
Number of  users (columns):  77


user_id,US041020210001,US061020210001,US071020210001,US191020210001,US191020210002,US191020210003,US191020210004,US191020210005,US191020210006,US191020210007,US191020210008,US191020210009,US191020210010,US191020210011,US191020210012,US251020210001,US251020210013,US251020210014,US251020210015,US251020210016,US251020210017,US251020210018,US251020210019,US251020210020,US271020210001,US271020210021,US271020210022,US271020210023,US271020210024,US271020210025,US271020210026,US271020210027,US271020210028,US271020210029,US271020210030,US271020210031,US271020210032,US271020210033,US271020210034,US271020210035,US271020210036,US271020210037,US281020210001,US281020210038,US281020210039,US281020210040,US281020210041,US281020210042,US281020210043,US281020210044,US281020210045,US281020210046,US281020210047,US281020210048,US281020210049,US281020210050,US281020210051,US281020210052,US281020210053,US281020210054,US281020210055,US281020210056,US281020210057,US281020210058,US281020210059,US281020210060,US281020210061,US281020210062,US281020210063,US281020210064,US281020210065,US281020210066,US281020210067,US281020210068,US281020210069,US281020210070,US281020210071
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PD041020210001,4.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0
PD041020210002,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.5,0.0,0.0,0.0,2.0,2.5,0.0,3.0,3.0,0.0,4.0,0.0,4.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210003,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0
PD041020210004,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,4.5,0.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210005,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,2.5,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,4.5,4.5,0.0,0.0,0.0,0.0,0.0
PD081020210006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
PD081020210007,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,1.5,0.0,0.0,0.0,1.5,1.5,0.0,0.0,1.5,0.0,0.0,0.0,1.0,2.0,2.5,2.0,0.0,0.0,2.0,0.0,3.0,3.0,0.0,0.0,0.0,4.5,4.5,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0
PD151020210001,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.5,0.0,4.0,0.0,4.5,5.0,5.0,0.0,0.0,0.0
PD161020210001,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [4]:
df_is_rated = df_users_products_ratings_pivotted.copy()
# df_is_rated.head(20)

column_names = df_users_products_ratings_pivotted.columns.values
print(column_names)

for name in column_names:
    df_is_rated.loc[df_is_rated[name] > 0, name] = 1

df_is_rated.head(20)

['US041020210001' 'US061020210001' 'US071020210001' 'US191020210001'
 'US191020210002' 'US191020210003' 'US191020210004' 'US191020210005'
 'US191020210006' 'US191020210007' 'US191020210008' 'US191020210009'
 'US191020210010' 'US191020210011' 'US191020210012' 'US251020210001'
 'US251020210013' 'US251020210014' 'US251020210015' 'US251020210016'
 'US251020210017' 'US251020210018' 'US251020210019' 'US251020210020'
 'US271020210001' 'US271020210021' 'US271020210022' 'US271020210023'
 'US271020210024' 'US271020210025' 'US271020210026' 'US271020210027'
 'US271020210028' 'US271020210029' 'US271020210030' 'US271020210031'
 'US271020210032' 'US271020210033' 'US271020210034' 'US271020210035'
 'US271020210036' 'US271020210037' 'US281020210001' 'US281020210038'
 'US281020210039' 'US281020210040' 'US281020210041' 'US281020210042'
 'US281020210043' 'US281020210044' 'US281020210045' 'US281020210046'
 'US281020210047' 'US281020210048' 'US281020210049' 'US281020210050'
 'US281020210051' 'US281020210052'

user_id,US041020210001,US061020210001,US071020210001,US191020210001,US191020210002,US191020210003,US191020210004,US191020210005,US191020210006,US191020210007,US191020210008,US191020210009,US191020210010,US191020210011,US191020210012,US251020210001,US251020210013,US251020210014,US251020210015,US251020210016,US251020210017,US251020210018,US251020210019,US251020210020,US271020210001,US271020210021,US271020210022,US271020210023,US271020210024,US271020210025,US271020210026,US271020210027,US271020210028,US271020210029,US271020210030,US271020210031,US271020210032,US271020210033,US271020210034,US271020210035,US271020210036,US271020210037,US281020210001,US281020210038,US281020210039,US281020210040,US281020210041,US281020210042,US281020210043,US281020210044,US281020210045,US281020210046,US281020210047,US281020210048,US281020210049,US281020210050,US281020210051,US281020210052,US281020210053,US281020210054,US281020210055,US281020210056,US281020210057,US281020210058,US281020210059,US281020210060,US281020210061,US281020210062,US281020210063,US281020210064,US281020210065,US281020210066,US281020210067,US281020210068,US281020210069,US281020210070,US281020210071
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PD041020210001,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
PD041020210002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210003,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
PD041020210004,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210005,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
PD081020210006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
PD081020210007,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
PD151020210001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
PD161020210001,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [5]:
def cal_cost_func(params, *args):
    Y, R, num_users, num_movies, num_features, lmbda = args
    # Unfold the U and W matrices from params
    X = params[0:num_movies*num_features].reshape(num_movies, num_features, \
               order = 'F')
    Theta = params[num_movies*num_features:].reshape(num_users, num_features, \
                  order = 'F')
    J_reg = (((Theta ** 2).sum()).sum() + sum(sum(X ** 2))) / 2 * lmbda
    J =  (((X.dot(Theta.T) * R - Y) ** 2).sum()).sum() / 2 + J_reg
    return J

In [6]:
def cal_gradient_decent(params, *args):
    Y, R, num_users, num_movies, num_features, lmbda = args
    # Unfold the U and W matrices from params
    X = params[0:num_movies*num_features].reshape(num_movies, num_features, \
               order = 'F')
    Theta = params[num_movies*num_features:].reshape(num_users, num_features, \
                  order = 'F')
    X_grad_reg = lmbda * X
    Theta_grad_reg = lmbda * Theta
    X_grad = (X.dot(Theta.T) * R - Y).dot(Theta) + X_grad_reg
    Theta_grad = ((X.dot(Theta.T) * R - Y).T).dot(X) + Theta_grad_reg
    grad = np.hstack([X_grad.flatten('F'), Theta_grad.flatten('F')])
    
    return grad

In [7]:
def normalize_rating(Y, R):
    m, n = np.shape(Y)
    Ymean = np.zeros((m, 1))
    Ynorm = np.zeros((m, n))
    for i in range(m):
        idx = np.nonzero(R[i, :] == 1.0)[0]
        Ymean[i] = Y[i, idx].mean(axis = 0)
        Ynorm[i, idx] = Y[i, idx] - Ymean[i]
    return Ymean, Ynorm

In [8]:
Y = df_users_products_ratings_pivotted.to_numpy()
R = df_is_rated.to_numpy()

print("Rating numpy array: ", Y)
print("Rating numpy array's shape: ", Y.shape)
print("Check rated numpy array\n", R)
print("Check rated numpy array's shape: \n", R.shape)

Rating numpy array:  [[4.  0.  2.5 ... 5.  5.  0. ]
 [2.5 0.  0.  ... 0.  0.  0. ]
 [4.  0.  0.  ... 5.  0.  5. ]
 ...
 [0.  0.  0.  ... 0.  5.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]
Rating numpy array's shape:  (61, 77)
Check rated numpy array
 [[1. 0. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Check rated numpy array's shape: 
 (61, 77)


In [9]:
Ymean, Ynorm = normalize_rating(Y, R)
num_products, num_users = np.shape(Y)
num_features = 3

X = np.random.standard_normal(num_products * num_features) 
Theta = np.random.standard_normal(num_users * num_features)

initial_parameters = np.hstack([X, Theta])
lmbda = 3
args = (Ynorm, R, num_users, num_products, num_features, lmbda)

theta = optimize.fmin_cg(cal_cost_func, initial_parameters, \
                         fprime=cal_gradient_decent, args=args, maxiter=100)

# Unfold the returned theta back into X and Theta
X = theta[0:num_products*num_features].reshape(num_products, num_features, \
           order = 'F')
Theta = theta[num_products*num_features:].reshape(num_users, num_features, \
              order = 'F')

Optimization terminated successfully.
         Current function value: 148.162436
         Iterations: 69
         Function evaluations: 109
         Gradient evaluations: 109


C:\Users\QUOCCU~1\AppData\Local\Temp/ipykernel_10512/663095750.py:7: RuntimeWarning: Mean of empty slice.
  Ymean[i] = Y[i, idx].mean(axis = 0)
c:\users\quoc cuong\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
# Make prediction
p = X.dot(Theta.T)
print("Shape of prediction array: ", p.shape)
print("Prediction Array: \n", p)

Shape of prediction array:  (61, 77)
Prediction Array: 
 [[ 0.09897775  0.17649551 -0.08984355 ...  1.36320298  1.43439526
   1.17710458]
 [ 0.08292571  0.17586317 -0.08879845 ...  1.3381376   1.42535456
   1.11814811]
 [ 0.11974262  0.12824527 -0.07311727 ...  1.12909795  1.14501849
   1.06023355]
 ...
 [ 0.06621885  0.13744451 -0.06798822 ...  1.02741448  1.09050235
   0.86878951]
 [ 0.07030966  0.1338519  -0.06803973 ...  1.02940131  1.08862282
   0.87695414]
 [ 0.00906672  0.02098774 -0.00993238 ...  0.15005043  0.15978516
   0.1263015 ]]


In [11]:
def get_product_name(product_id):
    query_str = f"select product_name from products p where product_id='{product_id}'"
    df_name= connector.query(query_str)
    return df_name.values.tolist()[0][0]

name = get_product_name('PD271020210019')
print(name)

Điện thoại OPPO Reno6 Pro 5G


In [12]:
# Get list of users haven't rated before
query_str = "select user_id from users where user_id not in (select user_id from ratings)"
df_results = connector.query(query_str)

print("Users haven't rated any products: \n", df_results)

Users haven't rated any products: 
           user_id
0  US251020210019
1  US251020210017
2  US251020210016
3  US251020210013
4  US251020210020
5  US281020210038
6  US251020210015
7  US251020210018
8  US251020210014
9  US191020210011


In [13]:
# Get the prediction of rating for the user
# order start from 0
import math
user_id = 'US251020210016'
user_index_in_matrix = df_users_products_ratings_pivotted.columns.get_loc(user_id)
print("Index of user in matrix: ", user_index_in_matrix)

user_prediction = p[:,user_index_in_matrix] + Ymean.flatten()
user_prediction = [x for x in user_prediction if str(x) != 'nan']

# Sort user prediction with descending order of score
sorted_indices = np.argsort(user_prediction)[::-1]
list_products = df_users_products_ratings_pivotted.index.values.tolist() 
n_top = 10
for position in range(n_top):
    index = sorted_indices[position]
    print(f"Product at Index: {index} - predict score: {user_prediction[index]} \nRecommend product: {get_product_name(list_products[index])}\n")

Index of user in matrix:  19
Product at Index: 23 - predict score: 5.0 
Recommend product: Điện thoại OPPO Reno6 Pro 5G

Product at Index: 21 - predict score: 4.5 
Recommend product: Điện Thoại Oppo Reno 6 5G (8GB/128G) - Hàng Chính Hãng

Product at Index: 22 - predict score: 4.5 
Recommend product: Điện Thoại Xiaomi Mi 11 Lite 5G NE 6GB l 128GB - Hàng Chính Hãng

Product at Index: 25 - predict score: 4.5 
Recommend product: Điện Thoại iPhone 12 Pro 256GB - Hàng Chính Hãng

Product at Index: 24 - predict score: 4.333333332919885 
Recommend product: Điện Thoại iPhone 13 Pro Max 1TB - Hàng Chính Hãng

Product at Index: 30 - predict score: 4.0 
Recommend product: Điện Thoại Xiaomi Mi 11 Lite 4G - Hàng Chính Hãng

Product at Index: 27 - predict score: 4.0 
Recommend product: Điện Thoại Samsung Galaxy Z Flip (8GB/256GB) - ĐÃ KÍCH HOẠT BẢO HÀNH ĐIỆN TỬ - Hàng Chính Hãng

Product at Index: 16 - predict score: 4.0 
Recommend product: Điện Thoại Vsmart Star 5 (4GB/64GB) - Hàng Chính Hãng

Produ